In [1]:
import re
import os
import numpy as np
import glob
from colorama import Fore, Back, Style
from tqdm import tqdm
import pandas as pd
from subprocess import Popen, PIPE, call, check_call
import datetime
import shutil
from datetime import datetime
import time

from help_functions import replace_gpr #user def
from lut import tests #user def 


## General defines, paths etc

In [279]:
transcript_path = os.environ['NOELV']
grlib_path = os.environ['GRLIB']
riscv_dv = os.environ['r_dv']+"/"

test_number = 4
print("Running the test: {}".format(tests[test_number]))
now = datetime.now()
current_time = now.strftime("%Y-%m-%d")
finish_after = int(1e5)


Running the test: riscv_rand_jump_test


## Generate RISCV-DV tests and run OVPsim
- Tests that work: 0
- Tests that sometimes timeout but sometimes doesn't: 1, 2, 10
- Tests that fail due to add instruction after **mret**: 3, 4
- Tests that doesn't run at all: 5, 6 (Never enters program after running the bootloader)
- Test 7 seems to outright fail
- Test 8,9 is maybe not applicable since ebreak is handled differently between ovpSIM and NOEL-V

In [280]:
param = "python3 run.py --custom_target target/rv64_noelv/ --iss ovpsim --simulator riviera --isa rv64gc --mabi lp64 --test "+tests[test_number]+" --iterations 1"
process = Popen('cd '+riscv_dv+' && source ~/.bashrc && module load aldec/riviera/2021.10 &&'+param, shell=True, stdout=PIPE, stderr=PIPE)
#process.wait()
stdout, stderr = process.communicate()
timeout = False
error = stderr.decode('ascii')

if(error.find("You do not have a valid license to run Riviera-PRO") == -1):
    path = glob.glob(riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.log")    
    with open(path[0],"r") as f:
        timeout = f.read().find("Info "+str(finish_after)+":") != -1
        f.close()
else:
    raise RuntimeError("License not found for Riviera PRO")

instr_cnt = finish_after+41+2 #+41 to account for bootloader, +2 for counter trailing
arg_timeout = "-gdisas=1 -gfinish_after_en=1 -gfinish_after="+str(instr_cnt)
arg_no_timeout = "-gdisas=1 -gfinish_after_en=0 -gfinish_after=0"
if(timeout):
    arg=arg_timeout
else:
    arg=arg_no_timeout
with open(grlib_path+"/designs/noelv-generic/Makefile","r") as f:
    txt = f.read()
    tmp = re.sub("(?<=VSIMOPT=).*",arg,txt)
    f.close()
    f = open(grlib_path+"/designs/noelv-generic/Makefile","w")
    f.write(tmp)
    f.close()

## Run RTL simulation
### For now only consider tests generated today
- How do we break simulation if we have a timeout in OVPsim?
    - It seems like execution time scales quite well to number of instructions*clock_period, add 20% extra sim time to compensate for IPC differences

In [ ]:
srec_tests = glob.glob(riscv_dv+"out_"+current_time+"/asm_test/"+tests[test_number]+"*.srec")
shutil.copy(srec_tests[0], transcript_path+'ram.srec') #copy the current test to the sim dir
print("Copied {} to the NOELV directory".format(srec_tests))

#launch sim
process = Popen('cd '+transcript_path+' && source ~/.bashrc && module add mentor/questasim/2021.3 && make sim-run', shell=True, stdout=PIPE, stderr=PIPE)
#process.wait()
stdout, stderr = process.communicate()
txt = stdout.decode('ascii')
if(txt.find("# ** Failure: *** IU in error mode, simulation halted ***") > -1 or txt.find("Test finished after") > -1 or txt.find("** Failure: Assertion violation.")):
    print("RTL Simulation Success")
else:
    print("RTL Simulation Failed")

if not os.path.exists(riscv_dv+"out_"+current_time+"/rtl_log/"):
    os.mkdir(riscv_dv+"out_"+current_time+"/rtl_log/")
shutil.copy(transcript_path+"transcript", riscv_dv+"out_"+current_time+"/rtl_log/"+tests[test_number]+"_"+re.search("\d*(?=.srec)",str(srec_tests)).group(0)+".log")

Copied ['/home/jonathanjonsson/MasterThesis/riscv-dv/out_2022-02-25/asm_test/riscv_rand_jump_test_0.srec'] to the NOELV directory


## Parse RTL transcript and create rtl_log

In [287]:
lookup = re.compile("#\s*\d*\s*ns\s*:\sC\d I\d : \d*\s*\[\d\] @0x[0-9a-fA-F]{16} \(0x[0-9a-fA-F]{4,8}\)\s*.*")
match = 0
f = open(transcript_path+"transcript")
transcript = f.readlines()
f.close()

#regex to extract content relevant to rtl sim
error_re = re.compile("[eE]rror | [wW]arning | [fF]ailure")
pc_csr_status_tval_re = re.compile("[0-9a-fA-F]{16}|X{16}")
binary_re = re.compile("(?<=(\(0x))[a0-z9]{4,8}(?=\))")
instruction_re = re.compile("\w{2,8}(?=(\s\w{2,3},))|(\w*\.[a-z])|nop|mret|ecall|fence|unknown instruction|ebreak")
instr_str_re = re.compile("\w*\s\w*,\s\w*,\s-?\w*(?=\s*W)|\w*(?=\s*W)|\w*\s\w*,\s\w*(?=\s*W)") #ugly
gpr_re = re.compile("(?<=([a-z]{2}\s))\w{2,3}(?=,)")
op1_re = re.compile("(?<=,\s)\w{1,8}((?=,)|(?=\s*W))|0\(\w{2,3}\)") 
op2_re = re.compile("(\w*,\s\w*,\s-?\w*)") #use split to get op2, don't have a nicer regex atm
csr_re = re.compile("(?<=\[)\w{2,3}(?=\s?=)")
mode_re = re.compile("(?<=PRV\[)\d*")
valid_re = re.compile("(?<=\[)\d(?=\]\s@)")
exception_re = re.compile("(?<=\[)\d(?=\]\sPRV)")

pc = []
binary = []
instr = []
instr_str = []
gpr = []
csr = []
gpr_val = []
op_0 = []
op_1 = []
status = []
tval = []
mode = []
exception = []
valid = []


for line in tqdm(transcript):
    if(re.search(lookup,line)): #check that line conforms to format then extract information
        valid.append(re.search(valid_re, line).group(0))
                
        pc_i = re.search(pc_csr_status_tval_re, line)
        if(pc_i):
            pc.append(pc_i.group(0))
        else:
            print(line)
            break
        binary.append(re.search(binary_re, line).group(0))
        instruction = re.search(instruction_re, line)
        if(instruction):
            instr.append(instruction.group(0))
        else:
            raise RuntimeError("Instruction not present in RegEx\n"+line)
        instr_str.append(re.search(instr_str_re, line).group(0))
        match3 = re.search(gpr_re, line)
        if(match3):
            gpr.append(match3.group(0))
        else:
            gpr.append("")
        match4 = re.search(op1_re, line)
        if(match4):
            op_0.append(match4.group(0))
        else:
            op_0.append("")
        match5 = re.search(op2_re,line)
        if(match5):
            op_1.append(match5.group(0).split(", ")[2])
        else:
            op_1.append("")
        match6 = re.search(csr_re,line)
        if(match6):
            csr.append(match6.group(0))
        else:
            print(line)
            raise RuntimeError("CSR RegEx not complete")

        match7 = re.findall(pc_csr_status_tval_re,line)
        if(len(match7) == 3):
            print(match7)
            print(line)
        gpr_val.append(match7[1])
        status.append(match7[2])
        tval.append(match7[3])
        mode.append(re.search(mode_re,line).group(0))
        exception.append(re.search(exception_re,line).group(0))



columns = ['pc','instr','gpr','csr','binary','mode','instr_str','operand','valid','exception']
rtl_log = pd.DataFrame(columns=columns)
rtl_log[columns[0]] = pc
rtl_log[columns[1]] = instr
rtl_log[columns[2]] = pd.Series(gpr) +":"+ pd.Series(gpr_val)
rtl_log[columns[3]] = csr
rtl_log[columns[4]] = binary
rtl_log[columns[5]] = mode
rtl_log[columns[6]] = instr_str
rtl_log[columns[7]] = pd.Series(gpr)+","+pd.Series(op_0)+","+pd.Series(op_1)
rtl_log[columns[8]] = valid
rtl_log[columns[9]] = exception
#log[columns[9]] = tval
rtl_log = rtl_log[41:].reset_index(drop=True) #41 is where the bootloader ends
def replace_gpr(pd_series,gpr,re_gpr):
    tmp = []
    for line in tqdm(pd_series):
        split_line = line.split(":")
        if(len(split_line[0]) > 0 and split_line[0] == gpr):
            split_line[0] = re_gpr
        ln = split_line[0]+":"+split_line[1]
        tmp.append(ln)
    return pd.Series(tmp)
rtl_log.gpr = replace_gpr(rtl_log.gpr, 'fp','s0') #replace fp with s0 to keep convention consistent
rtl_log['valid'] = rtl_log['valid'].astype('int32') #not true when we have an exception (ecall for instance)
#rtl_log = rtl_log[rtl_log.valid == 1].reset_index(drop=True) #only keep valid instructions
rtl_log = rtl_log[~((rtl_log['instr'] == "addi") & (rtl_log.shift(1)['instr'] == "mret") & (rtl_log['valid'] == 0))].reset_index(drop=True) #deal with gaislers addi injection after mret, this removes all of those lines


100%|██████████| 100758/100758 [00:00<00:00, 1018572.21it/s]


## Convert OVPsimlog to csv

In [288]:
logs = glob.glob(riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.log")
csvs = glob.glob(riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.csv")
for log in logs:
    process = Popen("python3 "+riscv_dv+"scripts/ovpsim_log_to_trace_csv.py --dont_truncate_after_first_ecall --log "+log+ " --csv "+log.strip(".log")+".csv", shell=True, stdout=PIPE, stderr=PIPE)
    stdout, stderr = process.communicate()
csvs = glob.glob(riscv_dv+"out_"+current_time+"/ovpsim_sim/"+tests[test_number]+"*.csv")
sim_log = pd.read_csv(csvs[0])
sim_log['mode'] = sim_log['mode'].astype('Int64')
sim_log['gpr'] = sim_log['gpr'].astype('str')
if(not timeout):
    sim_log = sim_log.truncate(after=len(sim_log)-2) #exit sequence slightly different between the rtl_sim and ovpsim

## Compare series to see if run was successfull

In [291]:
if(abs(len(sim_log)-len(rtl_log) < 1000)):
#if(abs(len(sim_log)-len(rtl_log) < 1000000)):

# This limit is somewhat arbitrary, not sure what best course of action is for different length logs.
# We can expect some difference in length when OVPsim timeouts since the runtime is then arbitrarily set, however, the rtl_log should be almost the same length
# as that of the sim_log. 
    
    if(len(sim_log) < len(rtl_log)):
        rtl_log=rtl_log.truncate(after=len(sim_log)-1)
        print("Truncating rtl_log to {} elements to meet size of sim_log".format(len(rtl_log)))
    elif(len(sim_log) == len(rtl_log)):
        print("Both logs are equally sized")

    else:
        sim_log=sim_log.truncate(after=len(rtl_log)-1)
        print("Truncating sim_log to {} elements to meet size of rtl_log".format(len(rtl_log)))
else:
    raise RuntimeError("Lengths of series differs with "+str(abs(len(sim_log)-len(rtl_log)))+" elements") 


matches = len(sim_log[(sim_log.gpr != "nan") & (rtl_log.gpr == sim_log.gpr)])
nan_count = len(sim_log[sim_log.gpr == "nan"])

tot_count = matches + nan_count
gpr_match = (tot_count == len(rtl_log.gpr))
binary_match = rtl_log.binary.eq(sim_log.binary).all()
pc_match = rtl_log.pc.eq(sim_log.pc).all()

if(gpr_match and binary_match and pc_match):
    print("GPR, Binary and PC contents match")
else:
    print("PC match: {}\nBinary match: {}\nGPR match: {} ".format(pc_match,binary_match,gpr_match))

Both logs are equally sized
PC match: True
Binary match: True
GPR match: False 


In [293]:
rtl_log[(sim_log.gpr != "nan") & (rtl_log.gpr != sim_log.gpr)]

,pc,instr,gpr,csr,binary,mode,instr_str,operand,valid,exception
248,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0
380,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0
512,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0
644,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0
776,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0
...,...,...,...,...,...,...,...,...,...,...
98588,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0
98720,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0
98852,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0
98984,00000000000021de,csrrs,a3:0000000000000800,a3,3b0026f3,3,"csrrs a3, pmpaddr0, x0","a3,pmpaddr0,x0",1,0


In [234]:
#bin(int(rtl_log[(sim_log.gpr != "nan") & (rtl_log.gpr != sim_log.gpr)].gpr.str.split(":")[247][1],16))

'0b100111110110'